In [1]:
import os
import numpy as np
from PIL import Image
import tensorflow as tf

# Load the TFLite model
model_path = "manipulation_detector_config_5.tflite"
interpreter = tf.lite.Interpreter(model_path=model_path)
interpreter.allocate_tensors()
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# Preprocesses an image before it is given to the model to ensure consistency
# across all images that the model is learning off of.
def preprocess_image(image_path):
    # Load the image and convert it to RGB colour space; 
    # we want all images to be in same colour space for consistency.
    with Image.open(image_path) as image:
        image = image.convert("RGB")
    image = image.resize((256, 256))
    # Convert the image to an array. Normalize its pixel values to be between 0 and 1.
    # Since most images in the training/test dataset are jpegs and in RGB, assume that
    # we can divide each pixel by 255 to normalize them to be between 0 and 1.
    preprocessed_image = np.array(image) / 255.0
    # Return the preprocessed image.
    return preprocessed_image

# Define a function to classify a single image
def classify_image(image_path):
    input_data = preprocess_image(image_path)
    input_data = np.expand_dims(input_data, axis=0).astype(np.float32)
    interpreter.set_tensor(input_details[0]['index'], input_data)
    interpreter.invoke()
    output_data = interpreter.get_tensor(output_details[0]['index'])
    return output_data[0][0]

# Define the folder where the images are located
folder_path = "reserve_2"

# Initialize counters for classification results
num_authentic = 0
num_manipulated = 0
num_au_authentic = 0
num_au_manipulated = 0
num_tp_authentic = 0
num_tp_manipulated = 0
num_au_filenames = 0
num_tp_filenames = 0

# Iterate over all the images in the folder
for filename in os.listdir(folder_path):
    if (filename.endswith(".jpg") or filename.endswith(".jpeg")):
        # Classify the image and update counters
        image_path = os.path.join(folder_path, filename)
        classification_result = classify_image(image_path)
        if classification_result < 0.5:
            print(f"{filename} - Manipulated: {classification_result}")
            num_manipulated += 1
            if (filename.startswith("Au") or filename.startswith("au")):
                num_au_manipulated += 1
                num_au_filenames += 1
            elif (filename.startswith("Tp") or filename.startswith("tp")):
                num_tp_manipulated += 1
                num_tp_filenames += 1
        else:
            print(f"{filename} - Authentic: {classification_result}")
            num_authentic += 1
            if (filename.startswith("Au") or filename.startswith("au")):
                num_au_authentic += 1
                num_au_filenames += 1
            elif (filename.startswith("Tp") or filename.startswith("tp")):
                num_tp_authentic += 1
                num_tp_filenames += 1

# Print the classification results and filename counts
print(f"Total authentic images: {num_authentic}")
print(f"Total manipulated images: {num_manipulated}")
print(f"Number of 'au' authentic images: {num_au_authentic}")
print(f"Number of 'au' manipulated images: {num_au_manipulated}")
print(f"Number of 'tp' authentic images: {num_tp_authentic}")
print(f"Number of 'tp' manipulated images: {num_tp_manipulated}")
print(f"Number of 'au' filenames: {num_au_filenames}")
print(f"Number of 'tp' filenames: {num_tp_filenames}")

precision = num_au_authentic/(num_au_authentic + num_tp_authentic)
recall = num_au_authentic/(num_au_authentic + num_au_manipulated)
accuracy = (num_au_authentic + num_tp_manipulated)/(num_au_authentic + num_tp_authentic + num_au_manipulated + num_tp_manipulated)

print(f"precision: {precision}")
print(f"recall: {recall}")
print(f"accuracy: {accuracy}")



Au_pla_30535.jpg - Manipulated: 0.46944478154182434
Tp_S_NRD_S_N_pla20018_pla20018_02489.jpeg - Manipulated: 0.42202576994895935
Tp_S_NNN_S_B_nat00029_nat00029_00645.jpeg - Manipulated: 0.00011780063505284488
Au_pla_30253.jpg - Manipulated: 0.18571867048740387
Au_pla_20040.jpg - Manipulated: 0.005990037694573402
Tp_S_NNN_S_N_arc00076_arc00076_01052.jpeg - Manipulated: 0.024668514728546143
Tp_S_NNN_S_N_cha00015_cha00015_00844.jpeg - Manipulated: 2.5869351247820305e-06
Au_pla_30247.jpg - Manipulated: 0.4900917708873749
Au_pla_20054.jpg - Manipulated: 0.49152037501335144
Tp_S_NNN_S_N_arc00027_arc00027_01095.jpeg - Manipulated: 0.014278565533459187
Au_pla_30521.jpg - Manipulated: 0.27382993698120117
Tp_S_NRN_S_N_pla00065_pla00065_01172.jpeg - Manipulated: 0.013765230774879456
Tp_S_NNN_S_N_art20027_art20027_01833.jpeg - Manipulated: 0.26667600870132446
Au_pla_30509.jpg - Authentic: 0.9878237247467041
Tp_S_NRN_S_N_ind00055_ind00055_00905.jpeg - Manipulated: 0.00013099281932227314
Au_pla_2006

Au_sec_00057.jpg - Authentic: 0.7022724747657776
Au_sec_30194.jpg - Manipulated: 0.01658567227423191
Tp_S_NNN_S_N_ind00063_ind00063_00909.jpeg - Manipulated: 0.00014436693163588643
Tp_S_NNN_S_N_arc10101_arc10101_11168.jpg - Manipulated: 0.11587090790271759
Au_sec_00043.jpg - Manipulated: 0.2079331874847412
Au_sec_30180.jpg - Authentic: 0.9454717636108398
Tp_S_NNN_S_N_sec20034_sec20034_01624.jpeg - Manipulated: 0.03342839702963829
Tp_S_NNN_S_N_art10010_art10010_20093.jpg - Manipulated: 0.32591667771339417
Au_nat_30389.jpg - Authentic: 0.891464352607727
Au_pla_30469.jpg - Authentic: 0.7985615730285645
Au_pla_30482.jpg - Authentic: 0.9230695366859436
Tp_S_NNN_S_B_nat20002_nat20002_02202.jpeg - Manipulated: 0.4293186068534851
Au_nat_30404.jpg - Authentic: 0.9849492907524109
Tp_S_NNN_S_B_sec20080_sec20080_01670.jpeg - Manipulated: 0.2517256438732147
Au_nat_30362.jpg - Authentic: 0.603682279586792
Tp_S_NNN_S_N_pla00031_pla00031_10985.jpg - Manipulated: 7.149035081965849e-05
Tp_S_NRN_S_B_nat2

Tp_S_NNN_S_O_ani00021_ani00021_20026.jpg - Manipulated: 0.013465861789882183
Au_pla_30736.jpg - Authentic: 0.9875298738479614
Tp_S_NNN_S_N_ani00071_ani00071_00827.jpeg - Authentic: 0.7289800047874451
Au_sec_30036.jpg - Manipulated: 0.4078499674797058
Au_nat_30559.jpg - Authentic: 0.9976455569267273
Tp_S_NNN_S_N_cha10181_cha10181_12296.jpg - Manipulated: 0.43259769678115845
Tp_S_NNN_S_N_nat20055_nat20055_02250.jpeg - Manipulated: 0.04066261276602745
Au_nat_20038.jpg - Manipulated: 0.0002516386448405683
Au_sec_30022.jpg - Authentic: 0.9478414058685303
Tp_S_NRN_S_N_cha10130_cha10130_12186.jpg - Manipulated: 0.4780104160308838
Tp_S_NNN_S_B_ind00056_ind00056_01345.jpeg - Manipulated: 0.0027531662490218878
Au_nat_20010.jpg - Manipulated: 0.2811421751976013
Au_nat_30203.jpg - Authentic: 0.9841903448104858
Tp_S_NNN_S_B_nat20001_nat20001_01505.jpeg - Manipulated: 0.3933185338973999
Tp_S_NRD_S_B_arc00047_arc00047_01079.jpeg - Manipulated: 0.04918524995446205
Au_pla_20096.jpg - Manipulated: 0.286

Tp_S_NNN_S_N_nat20047_nat20047_01551.jpeg - Manipulated: 0.2167520970106125
Au_pla_30126.jpg - Authentic: 0.9968070983886719
Au_nat_30188.jpg - Authentic: 0.9985529184341431
Au_sec_30381.jpg - Authentic: 0.9952661395072937
Tp_S_NRN_S_N_arc20065_arc20065_01705.jpeg - Manipulated: 0.17454494535923004
Tp_S_NNN_S_N_ani10222_ani10222_12395.jpg - Manipulated: 0.10130767524242401
Tp_S_NRN_S_N_cha20042_cha20042_02041.jpeg - Manipulated: 0.3372069299221039
Au_pla_30668.jpg - Authentic: 0.9217748045921326
Au_txt_00015.jpg - Manipulated: 0.11729103326797485
Tp_S_NNN_S_N_nat20053_nat20053_01557.jpeg - Manipulated: 0.17494581639766693
Tp_S_NNN_S_N_sec00077_sec00077_00752.jpeg - Manipulated: 0.0007050805725157261
Au_txt_00001.jpg - Manipulated: 0.23578666150569916
Au_sec_30395.jpg - Manipulated: 0.08836814016103745
Tp_S_NNN_S_B_sec10001_sec10001_20007.jpg - Manipulated: 0.44830143451690674
Tp_S_NNN_S_N_pla00011_pla00011_00550.jpeg - Manipulated: 1.6867604202275288e-08
Au_nat_30177.jpg - Authentic: 0

Tp_S_NNN_S_N_ind20012_ind20012_02271.jpeg - Manipulated: 0.2950845956802368
Au_nat_30610.jpg - Authentic: 0.9960353374481201
Au_pla_30696.jpg - Authentic: 0.9983301162719727
Tp_S_NNN_S_N_nat20026_nat20026_01530.jpeg - Manipulated: 0.18027275800704956
Tp_S_NNN_S_N_ind20050_ind20050_02296.jpeg - Manipulated: 0.31094658374786377
Au_sec_30419.jpg - Manipulated: 0.1605883240699768
Tp_S_NNN_S_N_ind00029_ind00029_00896.jpeg - Manipulated: 1.3777343849108448e-11
Au_sec_30394.jpg - Manipulated: 0.1521092802286148
Tp_S_NNN_S_N_nat20050_nat20050_02246.jpeg - Manipulated: 0.2729487419128418
Au_sec_30380.jpg - Manipulated: 0.047107599675655365
Tp_S_NNN_S_N_nat10107_nat10107_11348.jpg - Manipulated: 0.28033262491226196
Au_nat_30189.jpg - Authentic: 0.9986441135406494
Au_txt_00014.jpg - Authentic: 0.8402568697929382
Au_pla_30669.jpg - Authentic: 0.7421336770057678
Tp_S_NNN_S_N_ani00089_ani00089_11145.jpg - Manipulated: 2.445199243084062e-05
Au_pla_30641.jpg - Authentic: 0.6876125931739807
Au_pla_3012

...

Tp_S_NNN_S_N_arc20099_arc20099_02186.jpeg - Manipulated: 0.4562172293663025
Tp_S_NNN_S_N_arc00024_arc00024_01098.jpeg - Manipulated: 0.009962798096239567
Tp_S_NNN_S_N_arc00096_arc00096_01034.jpeg - Manipulated: 0.17861144244670868
Au_sec_00100.jpg - Authentic: 0.6850207448005676
Tp_S_NNN_S_B_txt00028_txt00028_20117.jpeg - Manipulated: 1.6281992429867387e-05
Au_pla_30258.jpg - Manipulated: 0.0003575864539016038
Au_pla_20063.jpg - Manipulated: 0.34507331252098083
Au_pla_30270.jpg - Manipulated: 0.1999930888414383
Tp_S_NNN_S_N_nat20029_nat20029_01533.jpeg - Manipulated: 0.24805326759815216
Tp_S_NRN_S_N_cha00064_cha00064_00383.jpeg - Manipulated: 0.000996037619188428
Tp_S_NRN_S_N_cha00059_cha00059_11029.jpg - Manipulated: 0.2761983871459961
Au_nat_30590.jpg - Authentic: 0.8561498522758484
Au_pla_30516.jpg - Authentic: 0.9433236122131348
Au_pla_30502.jpg - Authentic: 0.9556255340576172
Au_nat_30584.jpg - Authentic: 0.9186827540397644
Tp_S_NNN_S_N_arc20075_arc20075_01715.jpeg - Manipulated: 

Au_pla_30472.jpg - Authentic: 0.9994989633560181
Tp_S_NRN_S_N_sec00070_sec00070_10717.jpg - Manipulated: 0.0002974881499540061
Au_pla_30466.jpg - Authentic: 0.9951878786087036
Tp_S_NNN_S_N_ani20042_ani20042_02084.jpeg - Manipulated: 0.2628818154335022
Au_nat_30386.jpg - Authentic: 0.9927477240562439
Au_pla_30300.jpg - Authentic: 0.7790738344192505
Tp_S_NNN_S_N_arc10101_arc10101_11167.jpg - Manipulated: 0.11667656153440475
Au_sec_30164.jpg - Authentic: 0.9864116907119751
Tp_S_NNN_S_N_nat20065_nat20065_01569.jpeg - Manipulated: 0.0307228472083807
Tp_S_NNN_S_N_cha00073_cha00073_00861.jpeg - Manipulated: 0.004130221903324127
Au_sec_30602.jpg - Authentic: 0.9711818695068359
Au_pla_30499.jpg - Authentic: 0.9497027397155762
Tp_S_NRD_S_N_ani00100_ani00100_00220.jpeg - Manipulated: 0.003675904357805848
Au_sec_30616.jpg - Authentic: 0.9924102425575256
Tp_S_NNN_S_N_cha00008_cha00008_11157.jpg - Manipulated: 0.0006174274021759629
Au_nat_30379.jpg - Authentic: 0.9528679847717285
Tp_S_NRN_S_N_ani101

Au_nat_30230.jpg - Authentic: 0.9732123613357544
Au_nat_20023.jpg - Manipulated: 0.08294506371021271
Au_sec_30039.jpg - Authentic: 0.9844416379928589
Au_nat_30224.jpg - Authentic: 0.9922356605529785
Au_nat_20037.jpg - Manipulated: 0.0021945303305983543
Tp_S_NNN_S_N_nat00021_nat00021_11041.jpg - Manipulated: 5.45166949450504e-06
Au_nat_30542.jpg - Authentic: 0.7784210443496704
Tp_S_NRN_S_N_art20050_art20050_01856.jpeg - Manipulated: 0.4575512707233429
Au_sec_30005.jpg - Authentic: 0.803051233291626
Au_pla_20099.jpg - Manipulated: 0.30736038088798523
Au_nat_30218.jpg - Authentic: 0.9891276955604553
Tp_S_NRN_S_N_nat00034_nat00034_00954.jpeg - Manipulated: 1.0403129635960795e-05
Au_sec_30011.jpg - Manipulated: 0.3111208975315094
Tp_S_NNN_S_N_nat00050_nat00050_11066.jpg - Manipulated: 2.6101679395651445e-06
Au_nat_30595.jpg - Authentic: 0.8157663941383362
Au_nat_10155.jpeg - Authentic: 0.8425894379615784
Tp_S_NRN_S_N_arc20001_arc20001_01454.jpeg - Manipulated: 0.4670906364917755
Au_pla_3051

Au_nat_30144.jpg - Authentic: 0.841168999671936
Au_pla_00001.jpg - Authentic: 0.9990271329879761
Tp_S_NRN_S_N_arc00062_ani00058_11497.jpg - Manipulated: 0.00012191518908366561
Au_pla_00015.jpg - Authentic: 0.9560300707817078
Au_sec_30359.jpg - Authentic: 0.977704644203186
Tp_S_NNN_S_N_arc00035_arc00035_01089.jpeg - Manipulated: 0.011152124963700771
Tp_S_NNN_S_N_pla00098_pla00098_00619.jpeg - Manipulated: 1.921255261549959e-06
Au_nat_30150.jpg - Authentic: 0.999813437461853
Tp_S_NNN_S_N_nat10149_nat10149_12000.jpg - Manipulated: 0.3240125775337219
Au_nat_30636.jpg - Authentic: 0.9941532015800476
Tp_S_NRD_S_N_ind00085_ind00085_00482.jpeg - Manipulated: 0.005141600966453552
Tp_S_NNN_S_B_pla20037_pla20037_02384.jpeg - Manipulated: 0.15024812519550323
Tp_S_NNN_S_N_sec00076_sec00076_00774.jpeg - Manipulated: 0.20287440717220306
Au_sec_00073.jpg - Authentic: 0.6157296895980835
Tp_S_NRN_S_B_arc00053_arc00053_00273.jpeg - Manipulated: 8.045108756959962e-07
Tp_S_NNN_S_N_nat20010_nat20010_02210.j

Au_nat_30031.jpg - Authentic: 0.9726207256317139
Au_sec_20003.jpg - Manipulated: 0.27454113960266113
Au_sec_30210.jpg - Authentic: 0.9669380784034729
Tp_S_NRN_S_N_nat10147_nat10147_11995.jpg - Manipulated: 0.11510556191205978
Au_nat_30019.jpg - Authentic: 0.9769211411476135
Au_sec_30576.jpg - Authentic: 0.7407651543617249
Tp_S_NRN_S_N_nat00082_nat00082_00986.jpeg - Manipulated: 0.24986785650253296
Au_sec_30562.jpg - Authentic: 0.999647855758667
Tp_S_NRN_S_B_cha00086_art00012_00406.jpeg - Manipulated: 0.19130931794643402
Au_sec_20017.jpg - Manipulated: 0.0003200442879460752
Au_sec_30204.jpg - Authentic: 0.9998504519462585
Tp_S_NNN_S_B_cha00058_cha00058_00378.jpeg - Manipulated: 0.09168388694524765
Au_pla_30060.jpg - Authentic: 0.9731013178825378
Tp_S_NNN_S_N_arc10118_arc10118_10759.jpg - Manipulated: 0.21418379247188568
Au_pla_30706.jpg - Authentic: 0.9902700185775757
Au_sec_30589.jpg - Authentic: 0.9758631587028503
Au_pla_30712.jpg - Authentic: 0.8733711242675781
Au_cha_10137.jpg - Aut